In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Demo to Show Anywidget mode

In [2]:
import bigframes.pandas as bpd

This notebook demonstrates the anywidget display mode, which provides an interactive table experience.
Key features include:
- **Column Sorting:** Click on column headers to sort data in ascending, descending, or unsorted states.
- **Adjustable Column Widths:** Drag the dividers between column headers to resize columns.

In [3]:
bpd.options.bigquery.ordering_mode = "partial"
bpd.options.display.repr_mode = "anywidget"

Load Sample Data

In [4]:
df = bpd.read_gbq("bigquery-public-data.usa_names.usa_1910_2013")
print(df)

state gender  year     name  number
   AL      F  1910    Sadie      40
   AL      F  1910     Mary     875
   AR      F  1910     Vera      39
   AR      F  1910    Marie      78
   AR      F  1910  Lucille      66
   CA      F  1910 Virginia     101
   DC      F  1910 Margaret      72
   GA      F  1910  Mildred     133
   GA      F  1910     Vera      51
   GA      F  1910   Sallie      92
...

[5552452 rows x 5 columns]


Display Series in anywidget mode

In [5]:
test_series = df["year"]
print(test_series)

Computation deferred. Computation will process 44.4 MB


Display with Pagination

In [6]:
df

state,gender,year,name,number
AL,F,1910,Annie,482
AL,F,1910,Myrtle,104
AR,F,1910,Lillian,56
CT,F,1910,Anne,38
CT,F,1910,Frances,45
FL,F,1910,Margaret,53
GA,F,1910,Mae,73
GA,F,1910,Beatrice,96
GA,F,1910,Lola,47
IA,F,1910,Viola,49


### Sorting by Single-Column
You can sort the table by clicking on the headers of columns that have orderable data types (like numbers, strings, and dates). Non-orderable columns (like arrays or structs) do not have sorting controls.

**Sorting indicators (▲, ▼) are always visible for sorted columns. The unsorted indicator (●) is only visible when you hover over an unsorted column header.** The sorting control cycles through three states:
- **Unsorted (no indicator by default, ● on hover):** The default state. Click the header to sort in ascending order.
- **Ascending (▲):** The data is sorted from smallest to largest. Click again to sort in descending order.
- **Descending (▼):** The data is sorted from largest to smallest. Click again to return to the unsorted state.

### Adjustable Column Widths
You can easily adjust the width of any column in the table. Simply hover your mouse over the vertical dividers between column headers. When the cursor changes to a resize icon, click and drag to expand or shrink the column to your desired width. This allows for better readability and customization of your table view.

Programmatic Navigation Demo

In [7]:
from bigframes.display.anywidget import TableWidget
import math
  
# Create widget programmatically  
widget = TableWidget(df)
print(f"Total pages: {math.ceil(widget.row_count / widget.page_size)}")
  
# Display the widget
widget

Total pages: 555246


TableWidget(orderable_columns=['state', 'gender', 'year', 'name', 'number'], page_size=10, row_count=5552452, …

Test Navigation Programmatically

In [8]:
# Simulate button clicks programmatically
print("Current page:", widget.page)

# Go to next page
widget.page = 1
print("After next:", widget.page)

# Go to previous page
widget.page = 0
print("After prev:", widget.page)

Current page: 0
After next: 1
After prev: 0


Edge Case Demonstration

In [9]:
# Test with very small dataset
small_df = df.sort_values(["name", "year", "state"]).head(5)
small_widget = TableWidget(small_df)
print(f"Small dataset pages: {math.ceil(small_widget.row_count / small_widget.page_size)}")
small_widget

Small dataset pages: 1


TableWidget(orderable_columns=['state', 'gender', 'year', 'name', 'number'], page_size=10, row_count=5, table_…

### Displaying Generative AI results containing JSON
The `AI.GENERATE` function in BigQuery returns results in a JSON column. While BigQuery's JSON type is not natively supported by the underlying Arrow `to_pandas_batches()` method used in anywidget mode ([Apache Arrow issue #45262](https://github.com/apache/arrow/issues/45262)), BigQuery Dataframes automatically converts JSON columns to strings for display. This allows you to view the results of generative AI functions seamlessly.

In [10]:
bpd._read_gbq_colab("""
  SELECT
    AI.GENERATE(
      prompt=>(\"Extract the values.\", OBJ.GET_ACCESS_URL(OBJ.FETCH_METADATA(OBJ.MAKE_REF(gcs_path, \"us.conn\")), \"r\")),
      connection_id=>\"bigframes-dev.us.bigframes-default-connection\",
      output_schema=>\"publication_date string, class_international string, application_number string, filing_date string\") AS result,
    *
  FROM `bigquery-public-data.labeled_patents.extracted_data`
  LIMIT 5;
""")

/usr/local/google/home/shuowei/src/python-bigquery-dataframes/bigframes/dtypes.py:987: JSONDtypeWarning: JSON columns will be represented as pandas.ArrowDtype(pyarrow.json_())
instead of using `db_dtypes` in the future when available in pandas
(https://github.com/pandas-dev/pandas/issues/60958) and pyarrow.
  warnings.warn(msg, bigframes.exceptions.JSONDtypeWarning)


/usr/local/google/home/shuowei/src/python-bigquery-dataframes/bigframes/dtypes.py:987: JSONDtypeWarning: JSON columns will be represented as pandas.ArrowDtype(pyarrow.json_())
instead of using `db_dtypes` in the future when available in pandas
(https://github.com/pandas-dev/pandas/issues/60958) and pyarrow.
  warnings.warn(msg, bigframes.exceptions.JSONDtypeWarning)


result,gcs_path,issuer,language,publication_date,class_international,class_us,application_number,filing_date,priority_date_eu,representative_line_1_eu,applicant_line_1,inventor_line_1,title_line_1,number
"{'application_number': None, 'class_international': None, 'filing_date': None, 'publication_date': None, 'full_response': '{}', 'status': 'INVALID_ARGUMENT: Invalid field in objectref details, only a JSON object named gcs_metadata is allowed [type.googleapis.com/util.MessageSetPayload=\'[dremel.DremelErrorWithDetails] { argument_error { query_error { } } debug_info { error_message_template: ""Invalid field in objectref details, only a JSON object named $0 is allowed"" error_id: 3270173750 } }\']'}",gs://gcs-public-data--labeled-patents/espacenet_de5.pdf,EU,DE,03.10.2018,G06F 11/30,<NA>,18157347.8,19.02.2018,31.03.2017,Hoffmann Eitle,FUJITSU LIMITED,"Kukihara, Kensuke","METHOD EXECUTED BY A COMPUTER, INFORMATION PROCESSING APPARATUS AND",EP 3 382 553 A1
"{'application_number': None, 'class_international': None, 'filing_date': None, 'publication_date': None, 'full_response': '{}', 'status': 'INVALID_ARGUMENT: Invalid field in objectref details, only a JSON object named gcs_metadata is allowed [type.googleapis.com/util.MessageSetPayload=\'[dremel.DremelErrorWithDetails] { argument_error { query_error { } } debug_info { error_message_template: ""Invalid field in objectref details, only a JSON object named $0 is allowed"" error_id: 3270173750 } }\']'}",gs://gcs-public-data--labeled-patents/espacenet_de2.pdf,EU,DE,29.08.018,E04H 6/12,<NA>,18157874.1,21.02.2018,22.02.2017,Liedtke & Partner Patentanw√§lte,SHB Hebezeugbau GmbH,"VOLGER, Alexander",STEUERUNGSSYSTEM F√úR AUTOMATISCHE PARKH√ÑUSER,EP 3 366 869 A1
"{'application_number': None, 'class_international': None, 'filing_date': None, 'publication_date': None, 'full_response': '{}', 'status': 'INVALID_ARGUMENT: Invalid field in objectref details, only a JSON object named gcs_metadata is allowed [type.googleapis.com/util.MessageSetPayload=\'[dremel.DremelErrorWithDetails] { argument_error { query_error { } } debug_info { error_message_template: ""Invalid field in objectref details, only a JSON object named $0 is allowed"" error_id: 3270173750 } }\']'}",gs://gcs-public-data--labeled-patents/espacenet_de73.pdf,EU,DE,03.10.2018,H05B 6/12,<NA>,18165514.3,03.04.2018,30.03.2017,<NA>,BSH Hausger√§te GmbH,"Acero Acero, Jesus",VORRICHTUNG ZUR INDUKTIVEN ENERGIE√úBERTRAGUNG,EP 3 383 141 A2
"{'application_number': None, 'class_international': None, 'filing_date': None, 'publication_date': None, 'full_response': '{}', 'status': 'INVALID_ARGUMENT: Invalid field in objectref details, only a JSON object named gcs_metadata is allowed [type.googleapis.com/util.MessageSetPayload=\'[dremel.DremelErrorWithDetails] { argument_error { query_error { } } debug_info { error_message_template: ""Invalid field in objectref details, only a JSON object named $0 is allowed"" error_id: 3270173750 } }\']'}",gs://gcs-public-data--labeled-patents/espacenet_de70.pdf,EU,DE,03.10.2018,H01L 21/20,<NA>,18166536.5,16.02.2016,<NA>,"Scheider, Sascha et al",EV Group E. Thallner GmbH,"Kurz, Florian",VORRICHTUNG ZUM BONDEN VON SUBSTRATEN,EP 3 382 744 A1
"{'application_number': None, 'class_international': None, 'filing_date': None, 'publication_date': None, 'full_response': '{}', 'status': 'INVALID_ARGUMENT: Invalid field in objectref details, only a JSON object named gcs_metadata is allowed [type.googleapis.com/util.MessageSetPayload=\'[dremel.DremelErrorWithDetails] { argument_error { query_error { } } debug_info { error_message_template: ""Invalid field in objectref details, only a JSON object named $0 is allowed"" error_id: 3270173750 } }\']'}",gs://gcs-public-data--labeled-patents/espacenet_de56.pdf,EU,DE,03.10.2018,A01K 31/00,<NA>,18171005.4,05.02.2015,05.02.2014,Stork Bamberger Patentanw√§lte,Linco Food Systems A/S,"Thrane, Uffe",MASTH√ÑHNCHENCONTAINER ALS BESTANDTEIL EINER EINHEIT UND EINER ANORDNUNG,EP 3 381 276 A1
